# Hvor mange barn kommenterer på norske TikTok-byggeres profiler?

Dokumentasjon av metode til Skup-rapport 2022.  

=================

På forhånd har vi definert en liste over 50 av de største kroppsbyggerne i Norge. 

I tillegg har vi laget en database som består av følgende kolonner: 
* poster (navn på TikTok-brukeren som har laget videoen)
* poster_username (URL til TikTok-brukeren som har laget videoen)
* post_text (tekst som hører til TikTok-videoen)
* name (navn på TikTok-brukeren som har kommentert på videoen)
* username (URL til TikTok-brukeren som har kommentert på videoen))
* comment (kommentaren)

### Hente ut kommentarer fra TikTok.com

* Gå inn på en TikTok-konto
* Klikke bort cookie-melding
* Skrolle helt ned på siden
* Gå inn på den siste videoen

*OBS! TikTok endrer ofte den midterste verdien i class-ene, som for eksempel «tiktok-x6y88p-DivItemContainerV2». Bruk inspektør-verktøyet i nettleseren for å finne oppdatert info.* 

In [2]:
urls = {
    "USERNAME" : "https://www.tiktok.com/@USERNAME"
}

for poster,poster_username in urls.items():

    # Open Chromedriver and TikTok
    driver = webdriver.Chrome(options=chrome_options, service=Service(ChromeDriverManager().install()))
    driver.get(poster_username)
    driver.maximize_window()
    time.sleep(10)

    # Remove cookie banner
    driver.execute_script('''return document.querySelector('tiktok-cookie-banner').shadowRoot.querySelector('button[type="button"]')''').click()
    time.sleep(5)

    # Scroll down
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        time.sleep(5)
        new_height = driver.execute_script("return document.body.scrollHeight")
        time.sleep(5)
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(5)

     # Find number of videoes and click on last video
    print("Looper gjennom videoene fra bunn til topp")
    videos = driver.find_elements(By.CLASS_NAME, "tiktok-x6y88p-DivItemContainerV2")
    time.sleep(5)
    count = len(videos)
    driver.find_elements(By.CLASS_NAME, "tiktok-x6y88p-DivItemContainerV2")[-1].click()
    time.sleep(5)

* Hente ut kommentarer, navn og brukernavn
* Klikke videre til neste video

In [ ]:
# Browse all videos on page
while True:
    try: 
        post_text = driver.find_element(By.CSS_SELECTOR, "div.tiktok-x6f6za-DivContainer").text
        print(f"[*] Video nummer {x} av {count}")

        try: 
            comments = driver.find_elements(By.CSS_SELECTOR, "div.tiktok-16r0vzi-DivCommentContentContainer")
            time.sleep(2)
            if True: 
                for c in comments:
                    name = c.find_element(By.CSS_SELECTOR, "span.tiktok-mfqbp1-SpanUserNameText").text
                    username = c.find_element(By.CSS_SELECTOR, "a.tiktok-bhsg9d-StyledUserLinkName").get_attribute("href")
                    comment = c.find_element(By.CSS_SELECTOR, "p.tiktok-q9aj5z-PCommentText").text

                    # Add to DB
                    cur.execute("INSERT OR IGNORE INTO Brukere (poster, poster_username, post_text, name, username, comment) VALUES (?, ?, ?, ?, ?, ?)",
                    [poster, poster_username, post_text, name, username, comment])
                    conn.commit()                    

            time.sleep(2)

        except NoSuchElementException:
            break

        WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, "tiktok-1ps1qld-ButtonBasicButtonContainer-StyledVideoSwitch"))).click()
        time.sleep(5)

        x += 1

    except TimeoutException:
        print("Ingen flere videoer.")
        break

print(f"Ferdig med {poster}")

### Finne antall barn i alderen 9-14 år i datamaterialet + alle kommentarene de har skrevet

* SQL-spørring for å finne kommentarer der noen oppgir å være 9-14. Tar høyde for ulike måter å skrive alderen på. Alle treffene må gås igjennom manuelt for å luke ut kommentarer som viser til noe annet enn alder, eller som helt tydelig er tull.

In [ ]:
UPDATE Brukere
SET barn9_14 = "1"
WHERE (comment LIKE "% 9 år%" or comment like "% 10 år%" or comment like "% 11 år%" or comment like "% 12 år%" 
        or comment like "% 13 år%" or comment like "% 14 år%"
        or comment like "% 10år%" or comment like "% 11år%" or comment like "% 12år%" 
        or comment like "% 13år%" or comment like "% 14år%"
        or comment like "%ni år%" or comment like "%ti år%" or comment like "%elleve år%" 
        or comment like "%tolv år%" or comment like "%tretten år%" or comment like "%fjorten år%" 
        or comment like "%niår%" or comment like "%tiår%" or comment like "%elleveår%" 
        or comment like "%tolvår%" or comment like "%trettenår%" or comment like "%fjortenår%")
        AND username NOT IN 
        (SELECT poster_username FROM Brukere)

* Spørring for finne brukernavnene og gruppere dem slik at de bare kommer opp én gang. Dermed vet vi hvor mange det er snakk om.
* Lager også et nytt table med bare de som oppgir alder mellom 9 og 14 år.

In [ ]:
CREATE TABLE "Brukernavn9_14" AS
SELECT username FROM Brukere
WHERE barn9_14 = "1"
GROUP BY username;

* Bruke «inner join»-metoden for å koble tabelet med brukernavn med hele datamaterialet.

In [ ]:
SELECT Brukere.poster, Brukere.username, Brukere.comment FROM Brukere
INNER JOIN Brukernavn9_14 ON Brukernavn9_14.username = Brukere.username;